In [1]:
from utils import *

In [2]:
interact(plot_noise_effect, 
         plot_minmax=True, 
         noise_compositions=noise_compositions,
         metric = ["f1_weighted", "f1_macro", "accuracy", "recall", "precision"],
  #      savepath = "",
         task=results.Task.unique())
None

interactive(children=(SelectMultiple(description='Noise Composition', index=(0, 1), options=(['Ar'], ['Dr'], […